## Fixed Data:

In [1]:
pip install fredapi

In [2]:
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

## <font color='red'> It seems the risk-free rate is daily rate. is it in percentage or in unit 1? need to adjust to weekly rate and select Monday

In [3]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [4]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [5]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolListShort:
    print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    print('Merging Dataframes...')
    dfName = f"{i}_All"
    # change merge method to outer to ensure not missing any values.
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), locals()[frameName]) 
    
    # change merege method to left, to ensure all the records in api stored.
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate,how = 'left').sort_values(by=['Date'])

    print(f"{i} data successfully pulled\n")
print('\nALL DATA SUCCESSFULLY PULLED!')

Start Pulling BTC's data:
Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Merging Dataframes...
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Merging Dataframes...
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Merging Dataframes...
LTC data successfully pulled


ALL DATA SUCCESSFULLY PULLED!


## <font color='red'> Imputation for NA values -- to be resolved

## Calculate and Add Return to Dataframe (52 records for testing)

In [6]:
nainfo = pd.DataFrame(index=LTC_All.columns.values)  #can be improved

for i in symbolListShort:
     
    dfName = f"{i}_All" 

    temp = locals()[dfName][-53:-1].reset_index()
    
    locals()[dfName] = locals()[dfName][-52:].reset_index().drop('index', axis=1)
    
    natemp = locals()[dfName].isnull().sum().to_frame(name= i)
    
    #nainfo pick all the na values
    nainfo = nainfo.join(natemp)  #can be removed, just for data checking purpose
    
    
    locals()[dfName]['Return'] = (locals()[dfName]['Price'] - temp['Price'])/temp['Price']
    


In [7]:
LTC_All.head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Return
0,2021-04-26,269.060433,1.817989e+10,777669,1261380,0.02,0.201887
1,2021-05-03,385.704466,2.608153e+10,882361,1438322,0.02,0.433524
2,2021-05-10,296.493781,2.006374e+10,884372,1530001,0.02,-0.231293
3,2021-05-17,144.373194,9.776754e+09,788793,1305682,0.00,-0.513065
4,2021-05-24,170.925525,1.158400e+10,719597,1180854,0.01,0.183915


## Get LargeCap Sample(maket cap > 10000)

In [8]:
## create largeCapSample template

largeCapSample = pd.DataFrame(columns = LTC_All.columns.values)  # can be improved

largeCapSample['Asset'] = ''
print("Finding All Coins with MarketCap Greater than 10000")
## step 1: pick all maket cap > 10000
for i in symbolListShort:
     
    dfName = f"{i}_All" 
    
    ## find all date with MarketCap greater than 10000
    temp = locals()[dfName].query("MarketCap > 1000000")
    temp['Asset'] = i
    largeCapSample = largeCapSample.append(temp)
print("Done!")

Finding All Coins with MarketCap Greater than 10000
Done!


In [9]:
largeCapSample

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Return,Asset
0,2021-04-26,56613.202648,1.058532e+12,1861992.0,5778871,0.02,0.154432,BTC
1,2021-05-03,58218.566077,1.088991e+12,2038316.0,6622607,0.02,0.028357,BTC
2,2021-05-10,46406.331544,8.683518e+11,1908851.0,6062366,0.02,-0.202895,BTC
3,2021-05-17,34888.678313,6.530094e+11,1607054.0,4960283,0.00,-0.248191,BTC
4,2021-05-24,35639.981455,6.672647e+11,1508758.0,4981835,0.01,0.021534,BTC
...,...,...,...,...,...,...,...,...
47,2022-03-21,129.241438,9.039261e+09,748866.0,1514571,0.22,0.125519,LTC
48,2022-03-28,128.926292,9.023978e+09,807733.0,1566202,0.19,-0.002438,LTC
49,2022-04-04,111.343007,7.798857e+09,841901.0,1576883,0.18,-0.136382,LTC
50,2022-04-11,109.018201,7.641529e+09,832321.0,1652475,0.22,-0.020880,LTC


## <font color='red'> Need to be checked

## Reshape Dataframe

In [25]:
returnAll = largeCapSample.loc[:,['Date', 'Return', 'Asset']].pivot(index="Date", columns=["Asset"], values='Return').reset_index().rename_axis(None, axis=1).sort_values('Date')
returnAll.head()

,Date,BTC,ETH,LTC
0,2021-04-26,0.154432,0.276095,0.201887
1,2021-05-03,0.028357,0.331870,0.433524
2,2021-05-10,-0.202895,-0.089013,-0.231293
3,2021-05-17,-0.248191,-0.409063,-0.513065
4,2021-05-24,0.021534,0.128304,0.183915


## Calculate Momentum Matrix(return 1-week.)

In [11]:
momMatrix = returnAll.copy()
momMatrix['Date'] = momMatrix['Date'].shift(-1)
momMatrix = momMatrix.drop(momMatrix.tail(1).index)
momMatrix.head()

,Date,BTC,ETH,LTC
0,2021-05-03,0.154432,0.276095,0.201887
1,2021-05-10,0.028357,0.331870,0.433524
2,2021-05-17,-0.202895,-0.089013,-0.231293
3,2021-05-24,-0.248191,-0.409063,-0.513065
4,2021-05-31,0.021534,0.128304,0.183915


## Calculate MOM Percentile(30%, 70%)

In [13]:
# Only elect Assets(BTC, ETH...) columns
col = []
for colName in momMatrix.columns:
     if colName not in ['Date','MOM30p','MOM70p']:
         col.append(colName)
momMatrix[col].head()

,BTC,ETH,LTC
0,0.154432,0.276095,0.201887
1,0.028357,0.331870,0.433524
2,-0.202895,-0.089013,-0.231293
3,-0.248191,-0.409063,-0.513065
4,0.021534,0.128304,0.183915


In [15]:
# 30% pencentile
momMatrix[col].quantile(.3, axis=1).head()

0    0.182905
1    0.210465
2   -0.214254
3   -0.450664
4    0.085596
Name: 0.3, dtype: float64

In [16]:
momMatrix['MOM30p']=momMatrix[col].quantile(.3, axis=1)
momMatrix['MOM70p']=momMatrix[col].quantile(.7, axis=1)
momMatrix.head()

,Date,BTC,ETH,LTC,MOM30p,MOM70p
0,2021-05-03,0.154432,0.276095,0.201887,0.182905,0.231570
1,2021-05-10,0.028357,0.331870,0.433524,0.210465,0.372531
2,2021-05-17,-0.202895,-0.089013,-0.231293,-0.214254,-0.157342
3,2021-05-24,-0.248191,-0.409063,-0.513065,-0.450664,-0.344715
4,2021-05-31,0.021534,0.128304,0.183915,0.085596,0.150548


In [17]:
# reshape for later use
momPt = momMatrix[['MOM30p','MOM70p']]
momMatrix = momMatrix.drop(['MOM30p','MOM70p'], axis=1)

## Calculate Momentum portfolio returns

In [18]:
# delete the first 2 rows of returnAll to match the momMatrix date
returnAll = returnAll.drop(returnAll.head(1).index).reset_index().drop('index', axis=1)
returnAll.head()

,Date,BTC,ETH,LTC
0,2021-05-03,0.028357,0.331870,0.433524
1,2021-05-10,-0.202895,-0.089013,-0.231293
2,2021-05-17,-0.248191,-0.409063,-0.513065
3,2021-05-24,0.021534,0.128304,0.183915
4,2021-05-31,0.004418,0.134644,0.032219


In [19]:
# initialize df for low, medium and high MOM
lowMom = pd.DataFrame(columns = ['Date', 'LowMomentum'])
lowMom['Date'] = momMatrix['Date']

medMom = pd.DataFrame(columns = ['Date', 'MediumMomentum'])
medMom['Date'] = momMatrix['Date']

highMom = pd.DataFrame(columns = ['Date', 'HighMomentum'])
highMom['Date'] = momMatrix['Date']

In [20]:
# iterate through whole dataframe to get low, medium and high MOM
lowList = []
medList = []
highList = []
for i in range(0, momMatrix.shape[0]):
    for j in range(1, momMatrix.shape[1]):
        if momMatrix.iloc[i,j] < momPt.iloc[i,0]:
            lowList.append(returnAll.iloc[i,j])
        elif (momPt.iloc[i,0] < momMatrix.iloc[i,j]) and (momMatrix.iloc[i,j] < momPt.iloc[i,1]):
            medList.append(returnAll.iloc[i,j])
        elif momMatrix.iloc[i,j] > momPt.iloc[i,1]:
            highList.append(returnAll.iloc[i,j])
    lowMom.iloc[i,1] = sum(lowList) / len(lowList)
    medMom.iloc[i,1] = sum(medList) / len(medList)
    highMom.iloc[i,1] = sum(highList) / len(highList)

In [21]:
momLMH = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), [lowMom, medMom, highMom])
momLMH.head()

,Date,LowMomentum,MediumMomentum,HighMomentum
0,2021-05-03,0.028357,0.433524,0.33187
1,2021-05-10,-0.087269,0.172255,0.050289
2,2021-05-17,-0.229201,0.032106,-0.102829
3,2021-05-24,-0.125922,0.056156,-0.071738
4,2021-05-31,-0.099854,0.071853,-0.050947


## Calculate Momentum Factor

In [22]:
momFactor = pd.DataFrame(columns = ['Date', 'MOM_LTW'])
momFactor['Date'] = momMatrix['Date']
momFactor['MOM_LTW'] = momLMH['HighMomentum'] - momLMH['LowMomentum']

In [23]:
momFactor.head(20)

,Date,MOM_LTW
0,2021-05-03,0.303513
1,2021-05-10,0.137558
2,2021-05-17,0.126372
3,2021-05-24,0.054184
4,2021-05-31,0.048908
5,2021-06-07,0.013285
6,2021-06-14,0.014183
7,2021-06-21,0.023543
8,2021-06-28,0.012332
9,2021-07-05,0.006211
